In [ ]:
import pandas as pd

In [ ]:
authors = pd.read_csv("/kaggle/input/ods-ai-hacktahon/participants/data/authors.csv")
candidates = pd.read_csv("/kaggle/input/ods-ai-hacktahon/participants/submit/candidates.csv") #No clue
book_genres = pd.read_csv("/kaggle/input/ods-ai-hacktahon/participants/data/book_genres.csv")
editions = pd.read_csv("/kaggle/input/ods-ai-hacktahon/participants/data/editions.csv")
genres = pd.read_csv("/kaggle/input/ods-ai-hacktahon/participants/data/genres.csv")
interactions = pd.read_csv("/kaggle/input/ods-ai-hacktahon/participants/data/interactions.csv")
users = pd.read_csv("/kaggle/input/ods-ai-hacktahon/participants/data/users.csv")

In [ ]:
data = [authors, book_genres, editions, genres, interactions, users]
data_names = ['authors', 'book_genres', 'editions', 'genres', 'interactions', 'users']
for i in range(len(data)): 
    print(f"{data_names[i]}")
    print(data[i].columns.to_list())
    print()

In [ ]:
book_data = book_genres.merge(genres, on='genre_id')
editions = editions.merge(book_data, on='book_id')
editions = editions.merge(authors, on='author_id')
interactions = interactions.merge(editions, on='edition_id')
interactions = interactions.merge(editions, on='edition_id')
data = interactions.merge(users, on='user_id')